In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import pyLDAvis as plda
import pyLDAvis.gensim as plgen
import re
import gensim.corpora as gcp
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.hdpmodel import HdpModel
import nlpaug.augmenter.word as naw


## 1. Data Aquisition

In [2]:
df = pd.read_excel("merged_data-v1.1.xlsx")
df

,company,companyOverviewLink,companyRating,companyReviewCount,normTitle,extractedSalary/max,extractedSalary/min,extractedSalary/type,formattedLocation,formattedRelativeTime,jobDescription,snippet,taxonomyAttributes/1/attributes/0/label,taxonomyAttributes/4/attributes/0/label,viewJobLink
0,Datamatics - RJ Globus Solution Inc,https://www.indeed.com/cmp/Datamatics-2aefac7a...,3.2,29.0,Admin Specialist,25000.0,20000.0,monthly,Cebu City,2024-11-20,Key responsibilities\n\n Provide general admin...,Perform data entry and scan documents.\n Provi...,8 hour shift,Permanent,/viewjob?jk=d5d17ccb99a1f2af&from=vjs&tk=1id50...
1,BPO Caldwell North,https://www.indeed.com/cmp/Caldwell-Communicat...,NaN,NaN,Data Entry Clerk,29000.0,20000.0,monthly,Manila,2024-11-20,"For regular employees, we offer an industry-le...",HMO with 2 free beneficiaries on Day One.\n Sh...,8 hour shift,Full-time,/viewjob?jk=063947cf48e0371b&from=vjs&tk=1id50...
2,Hunters Hub Incorporated,https://www.indeed.com/cmp/Hunters-Hub-Incorpo...,4.0,2.0,Business Analyst,70000.0,70000.0,monthly,Makati,2024-11-20,CAN START ASAP\n\nQualifications\n\n Have pers...,Have personally led and implemented initiative...,8 hour shift,Full-time,/viewjob?jk=7d374cddff0567a6&from=vjs&tk=1id50...
3,Ben Edictio Corporated,https://www.indeed.com/cmp/Ben-Edictio-Corpora...,NaN,NaN,SQL Developer,55000.0,40000.0,monthly,Mandaluyong,2024-11-20,This is minimum of 12 Months Project Based emp...,This is minimum of 12 Months Project Based emp...,Day shift,Full-time,/viewjob?jk=21151b1b674fcc6d&from=vjs&tk=1id50...
4,Orbit Teleservices,https://www.indeed.com/cmp/Orbit-Solutions-Cen...,NaN,NaN,Data Entry Clerk,30000.0,18000.0,monthly,Marikina,2024-11-20,WANT TO GET HIRED ASAP? GRAB OUR JOBS TODAY AN...,Non-Voice Agent (limited slots).\n Amenable to...,8 hour shift,Permanent,/viewjob?jk=a38ebf6c13f2858f&from=vjs&tk=1id50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,"Advanced Diagnostic Veterinary Laboratories, Inc.",https://www.indeed.com/cmp/American-Bio--clini...,4.0,3.0,Accounts Payable Clerk,49000.0,44000.0,monthly,Alabang,2024-11-28,"Location: Alabang, Muntinlupa, PhilippinesRepo...",Strong attention to detail and accuracy in fin...,Evening shift,Permanent,/viewjob?jk=50dd7ca27edbb322&from=vjs&tk=1iduf...
1117,Data Lake Inc,https://www.indeed.com/cmp/Data-Lake-Inc-1?cam...,NaN,NaN,Network Engineer,35000.0,25000.0,monthly,Manila,2024-11-28,JOB OVERVIEW:\nAs a Network and Data Engineer ...,"Proficiency in database design, data modeling,...",8 hour shift,Permanent,/viewjob?jk=ef2ea4ed93d34ca9&from=vjs&tk=1iduf...
1118,CDC Manufacturing Corporation (Melawares),https://www.indeed.com/cmp/Cdc-Manufacturing-C...,NaN,NaN,Data Entry Clerk,-1.0,525.0,daily,Meycauayan,2024-11-28,College Graduate of any course\n Computer Lite...,Maintain and update existing data records as n...,8 hour shift,Permanent,/viewjob?jk=212481fe1add0afb&from=vjs&tk=1iduf...
1119,TechTiera Corporation,https://www.indeed.com/cmp/Techtiera-Corporati...,NaN,NaN,Data Conversion Operator,-1.0,100000.0,monthly,Manila,2024-11-28,Work Location : BGC\nWork Schedule : WFH until...,"Experience with data warehousing, data archite...",Day shift,Full-time,/viewjob?jk=23d4d8ec48428bb6&from=vjs&tk=1iduf...


# 1.2 EDA

In [3]:
df["normTitle"].unique()

array(['Admin Specialist', 'Data Entry Clerk', 'Business Analyst',
       'SQL Developer', 'Data Analyst', 'Developer',
       'Marketing Specialist', 'Engagement Specialist',
       'Financial Analyst', 'Inventory Specialist',
       'Operation Specialist', 'Data Specialist', 'Automation Specialist',
       'IT Analyst', 'Inventory Analyst', 'Tableau Developer',
       'Market Researcher', 'Data Center Technician', 'Demand Planner',
       'Oracle PL SQL Developer', 'Marketing Manager', 'Data Processor',
       'Social Media Specialist', 'Data Engineer', 'Data Steward',
       'Senior Data Steward', 'Data Protection Officer',
       'E-commerce Specialist', 'Virtual Assistant', 'Software Engineer',
       'Business Process Analyst', 'Digital Marketer',
       'Supply Chain Analyst', 'Junior Data Scientist',
       'Financial Accountant', 'Tutor',
       'Data Analsyt (sql) - Up to 75k Monthly',
       'Administrative Assistant', 'Customer Service/data Entry',
       'Vehicle Technicia

In [4]:
df.describe()

,companyRating,companyReviewCount,extractedSalary/max,extractedSalary/min,formattedRelativeTime
count,291.000000,291.000000,1121.000000,1121.000000,1121
mean,3.663918,60.288660,37734.130687,29342.447342,2024-11-24 09:59:53.577163264
min,1.000000,2.000000,-1.000000,0.000000,2024-11-19 00:00:00
25%,3.300000,4.000000,20000.000000,18000.000000,2024-11-22 00:00:00
50%,3.900000,13.000000,30000.000000,22000.000000,2024-11-25 00:00:00
75%,4.400000,33.000000,50000.000000,40000.000000,2024-11-27 00:00:00
max,5.000000,4199.000000,664000.000000,531000.000000,2024-11-30 00:00:00
std,0.852767,351.676153,37480.900007,29008.495727,NaN


In [5]:
df.isna().sum()

company                                      0
companyOverviewLink                          0
companyRating                              830
companyReviewCount                         830
normTitle                                    0
extractedSalary/max                          0
extractedSalary/min                          0
extractedSalary/type                         0
formattedLocation                            0
formattedRelativeTime                        0
jobDescription                               0
snippet                                      0
taxonomyAttributes/1/attributes/0/label    140
taxonomyAttributes/4/attributes/0/label      0
viewJobLink                                  0
dtype: int64

In [6]:
df.columns

Index(['company', 'companyOverviewLink', 'companyRating', 'companyReviewCount',
       'normTitle', 'extractedSalary/max', 'extractedSalary/min',
       'extractedSalary/type', 'formattedLocation', 'formattedRelativeTime',
       'jobDescription', 'snippet', 'taxonomyAttributes/1/attributes/0/label',
       'taxonomyAttributes/4/attributes/0/label', 'viewJobLink'],
      dtype='object')

In [7]:
df_column_clean = df.drop(["companyOverviewLink","viewJobLink",'companyRating', 'companyReviewCount',"taxonomyAttributes/1/attributes/0/label"],axis=1).copy()
df_column_clean.head()

,company,normTitle,extractedSalary/max,extractedSalary/min,extractedSalary/type,formattedLocation,formattedRelativeTime,jobDescription,snippet,taxonomyAttributes/4/attributes/0/label
0,Datamatics - RJ Globus Solution Inc,Admin Specialist,25000.0,20000.0,monthly,Cebu City,2024-11-20,Key responsibilities\n\n Provide general admin...,Perform data entry and scan documents.\n Provi...,Permanent
1,BPO Caldwell North,Data Entry Clerk,29000.0,20000.0,monthly,Manila,2024-11-20,"For regular employees, we offer an industry-le...",HMO with 2 free beneficiaries on Day One.\n Sh...,Full-time
2,Hunters Hub Incorporated,Business Analyst,70000.0,70000.0,monthly,Makati,2024-11-20,CAN START ASAP\n\nQualifications\n\n Have pers...,Have personally led and implemented initiative...,Full-time
3,Ben Edictio Corporated,SQL Developer,55000.0,40000.0,monthly,Mandaluyong,2024-11-20,This is minimum of 12 Months Project Based emp...,This is minimum of 12 Months Project Based emp...,Full-time
4,Orbit Teleservices,Data Entry Clerk,30000.0,18000.0,monthly,Marikina,2024-11-20,WANT TO GET HIRED ASAP? GRAB OUR JOBS TODAY AN...,Non-Voice Agent (limited slots).\n Amenable to...,Permanent


# 2. Text Pre-processing

In [8]:
df_word_column = df["snippet"].copy().to_list()
df_word_column[:10]

['Perform data entry and scan documents.\n Provide general administrative and clerical support.\n Assist in resolving administrative concerns.',
 'HMO with 2 free beneficiaries on Day One.\n Shift Schedule (Day shift/Mid shift/Night shift).\n Pioneer, Non-voice, and Easy Accounts Available.',
 'Have personally led and implemented initiatives or projects especially related to digital channels or platforms and have dealt with program managers, project…',
 'This is minimum of 12 Months Project Based employment : It is still a Full time job that you will be having a full-time schedule but as project employee, *it is…',
 'Non-Voice Agent (limited slots).\n Amenable to working onsite(office-based).\n Filipino Citizen, or hold relevant residence status.\n 18 years old and above.',
 'A Data Entry Specialist is responsible for inputting, updating, and maintaining accurate data in various computer systems and databases.',
 'Non-Voice Agent (limited slots).\n Amenable to working onsite(office-bas

# 3. Data Augmentation

In [9]:
df_randomize = pd.read_excel("merged_data-v1.1.xlsx").sample(n= int(np.floor(len(df)*0.1))).reset_index(drop=True)
snippet_info = naw.synonym.SynonymAug(aug_src='wordnet')
synonym_comments = [snippet_info.augment(sentence)[0] for sentence in df_randomize["snippet"].to_list()]

df_augment = df_randomize.copy()
df_augment["snippet"] = synonym_comments


len(df_augment)

112

In [10]:
df_merged = pd.concat([df,df_augment]).reset_index(drop=True)
df_merged

,company,companyOverviewLink,companyRating,companyReviewCount,normTitle,extractedSalary/max,extractedSalary/min,extractedSalary/type,formattedLocation,formattedRelativeTime,jobDescription,snippet,taxonomyAttributes/1/attributes/0/label,taxonomyAttributes/4/attributes/0/label,viewJobLink
0,Datamatics - RJ Globus Solution Inc,https://www.indeed.com/cmp/Datamatics-2aefac7a...,3.2,29.0,Admin Specialist,25000.0,20000.0,monthly,Cebu City,2024-11-20,Key responsibilities\n\n Provide general admin...,Perform data entry and scan documents.\n Provi...,8 hour shift,Permanent,/viewjob?jk=d5d17ccb99a1f2af&from=vjs&tk=1id50...
1,BPO Caldwell North,https://www.indeed.com/cmp/Caldwell-Communicat...,NaN,NaN,Data Entry Clerk,29000.0,20000.0,monthly,Manila,2024-11-20,"For regular employees, we offer an industry-le...",HMO with 2 free beneficiaries on Day One.\n Sh...,8 hour shift,Full-time,/viewjob?jk=063947cf48e0371b&from=vjs&tk=1id50...
2,Hunters Hub Incorporated,https://www.indeed.com/cmp/Hunters-Hub-Incorpo...,4.0,2.0,Business Analyst,70000.0,70000.0,monthly,Makati,2024-11-20,CAN START ASAP\n\nQualifications\n\n Have pers...,Have personally led and implemented initiative...,8 hour shift,Full-time,/viewjob?jk=7d374cddff0567a6&from=vjs&tk=1id50...
3,Ben Edictio Corporated,https://www.indeed.com/cmp/Ben-Edictio-Corpora...,NaN,NaN,SQL Developer,55000.0,40000.0,monthly,Mandaluyong,2024-11-20,This is minimum of 12 Months Project Based emp...,This is minimum of 12 Months Project Based emp...,Day shift,Full-time,/viewjob?jk=21151b1b674fcc6d&from=vjs&tk=1id50...
4,Orbit Teleservices,https://www.indeed.com/cmp/Orbit-Solutions-Cen...,NaN,NaN,Data Entry Clerk,30000.0,18000.0,monthly,Marikina,2024-11-20,WANT TO GET HIRED ASAP? GRAB OUR JOBS TODAY AN...,Non-Voice Agent (limited slots).\n Amenable to...,8 hour shift,Permanent,/viewjob?jk=a38ebf6c13f2858f&from=vjs&tk=1id50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,Orbit NCR III,https://www.indeed.com/cmp/Orbit-Ncr-III?campa...,NaN,NaN,Data Entry Clerk,33000.0,18500.0,monthly,Santa Mesa,2024-11-22,WANT TO GET HIRED ASAP? GRAB OUR JOBS TODAY AN...,Non - Vocalism Agent (limited slot ). Amenable...,NaN,Permanent,/viewjob?jk=7d1bf618d28dd14b&from=vjs&tk=1id9r...
1229,WATSONS PERSONAL CARE STORES (PHILS.) INC.,https://www.indeed.com/cmp/Watsons-Personal-Ca...,1.4,5.0,Inventory Associate,560.0,560.0,daily,Dasmariñas,2024-11-28,BE PART OF WATSONS:Investors in People (IIP) A...,Our Depot Inventorying Shop assistant be respo...,8 hour shift,Permanent,/viewjob?jk=d4753e61682a41f3&from=vjs&tk=1idpa...
1230,iFive Global,https://www.indeed.com/cmp/Ifive-Global?campai...,3.6,12.0,Business Analyst,58000.0,58000.0,monthly,Pasig,2024-11-22,Highlights \n· Competitive compensation and be...,Competitive compensation and benefits software...,Night shift,Full-time,/viewjob?jk=12b2980137bffa1d&from=vjs&tk=1idce...
1231,DFNN Inc.,https://www.indeed.com/cmp/Dfnn?campaignid=mob...,NaN,NaN,Senior Business Analyst,50000.0,25000.0,monthly,Taguig,2024-11-22,Key Responsibilities:\n\n Requirements Gatheri...,"Proficiency in data analysis tools, project ma...",8 hour shift,Full-time,/viewjob?jk=04e80d59e78756fb&from=vjs&tk=1idf0...


# 3.1 Text Processing Part 2

In [11]:
df_word_comments = [re.split(r"[,\s\t;.\n/]+",sentence) for sentence in df_merged["snippet"].copy().to_list()]
df_word_comments_cleaned = list()
for sentence in df_word_comments:
    df_word_comments_cleaned.append([word.lower() for word in sentence if word != ''])
df_word_comments

[['Perform',
  'data',
  'entry',
  'and',
  'scan',
  'documents',
  'Provide',
  'general',
  'administrative',
  'and',
  'clerical',
  'support',
  'Assist',
  'in',
  'resolving',
  'administrative',
  'concerns',
  ''],
 ['HMO',
  'with',
  '2',
  'free',
  'beneficiaries',
  'on',
  'Day',
  'One',
  'Shift',
  'Schedule',
  '(Day',
  'shift',
  'Mid',
  'shift',
  'Night',
  'shift)',
  'Pioneer',
  'Non-voice',
  'and',
  'Easy',
  'Accounts',
  'Available',
  ''],
 ['Have',
  'personally',
  'led',
  'and',
  'implemented',
  'initiatives',
  'or',
  'projects',
  'especially',
  'related',
  'to',
  'digital',
  'channels',
  'or',
  'platforms',
  'and',
  'have',
  'dealt',
  'with',
  'program',
  'managers',
  'project…'],
 ['This',
  'is',
  'minimum',
  'of',
  '12',
  'Months',
  'Project',
  'Based',
  'employment',
  ':',
  'It',
  'is',
  'still',
  'a',
  'Full',
  'time',
  'job',
  'that',
  'you',
  'will',
  'be',
  'having',
  'a',
  'full-time',
  'schedule'

# 4. Feature Engineering

In [12]:
id2word = gcp.Dictionary(df_word_comments_cleaned)
corpus = [id2word.doc2bow(text) for text in df_word_comments]
corpus

[[(0, 2),
  (1, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (12, 1),
  (13, 1),
  (14, 1)],
 [(1, 1), (16, 1), (19, 1), (22, 1), (27, 1), (31, 2), (32, 1), (33, 1)],
 [(1, 2),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1)],
 [(30, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (62, 1),
  (63, 1),
  (64, 2),
  (65, 1),
  (67, 1),
  (68, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (75, 1),
  (76, 1),
  (77, 1)],
 [(1, 1),
  (43, 1),
  (50, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1),
  (92, 1),
  (93, 1)],
 [(1, 2),
  (5, 1),
  (9, 1),
  (64, 1),
  (94, 1),
  (95, 1),
  (96, 1),
  (97, 1),
  (98, 1),
  (99, 1),
  (100, 1),
  (102, 1),
  (103, 1),
  (104, 1)],
 [(1, 1),
  (43, 1

In [13]:
corpus[:1]

[[(0, 2),
  (1, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (12, 1),
  (13, 1),
  (14, 1)]]

In [14]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:3]]

[[('administrative', 2),
  ('and', 2),
  ('clerical', 1),
  ('concerns', 1),
  ('data', 1),
  ('documents', 1),
  ('entry', 1),
  ('general', 1),
  ('in', 1),
  ('resolving', 1),
  ('scan', 1),
  ('support', 1)],
 [('and', 1),
  ('2', 1),
  ('beneficiaries', 1),
  ('free', 1),
  ('on', 1),
  ('shift', 2),
  ('shift)', 1),
  ('with', 1)],
 [('and', 2),
  ('with', 1),
  ('channels', 1),
  ('dealt', 1),
  ('digital', 1),
  ('especially', 1),
  ('have', 1),
  ('implemented', 1),
  ('initiatives', 1),
  ('led', 1),
  ('managers', 1),
  ('or', 2),
  ('personally', 1),
  ('platforms', 1),
  ('program', 1),
  ('projects', 1),
  ('project…', 1),
  ('related', 1),
  ('to', 1)]]

# Creating ML model

In [15]:
lda_modoel = LdaModel(corpus=corpus,
                      id2word=id2word,
                      num_topics=20,
                      chunksize=100,
                      alpha='auto',
                      per_word_topics=True)
print(lda_modoel.print_topics())
doc_lda = lda_modoel[corpus]



''

[(0, '0.164*"data" + 0.155*"and" + 0.052*"to" + 0.031*"with" + 0.028*"systems" + 0.025*"in" + 0.025*"analyze" + 0.023*"financial" + 0.023*"of" + 0.021*"documentation"'), (1, '0.061*"have" + 0.054*"manner" + 0.049*"in" + 0.046*"if" + 0.045*"of" + 0.045*"with" + 0.043*"a" + 0.041*"and" + 0.039*"you" + 0.039*"to"'), (2, '0.202*"reports" + 0.179*"from" + 0.045*"new" + 0.036*"processes" + 0.031*"generate" + 0.014*"to" + 0.012*"and" + 0.011*"data" + 0.009*"dashboards" + 0.009*"raw"'), (3, '0.118*"-" + 0.083*"relevant" + 0.077*"to" + 0.066*"or" + 0.065*"above" + 0.064*"and" + 0.061*"years" + 0.053*"working" + 0.050*"old" + 0.049*"18"'), (4, '0.109*"of" + 0.101*"30" + 0.094*"processing" + 0.090*"skills" + 0.090*"typing" + 0.061*"wpm" + 0.025*"needed" + 0.017*"or" + 0.016*"and" + 0.016*"above"'), (5, '0.173*"and" + 0.150*"data" + 0.066*"to" + 0.044*"the" + 0.036*"…" + 0.032*"with" + 0.029*"of" + 0.026*"for" + 0.025*"in" + 0.014*"administrative"'), (6, '0.142*"shift" + 0.140*"on" + 0.090*"with" 

''

In [16]:
plda.enable_notebook()
p = plgen.prepare(lda_modoel, corpus, id2word)
p


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.288023 -0.139171       1        1  30.591519
0      0.233416 -0.134303       2        1  12.474094
3      0.157063  0.316127       3        1   9.603198
11     0.140875  0.094460       4        1   6.735018
1      0.133313 -0.088583       5        1   6.468600
9      0.147362  0.158132       6        1   6.120439
12     0.111701 -0.087355       7        1   4.922766
17     0.050552  0.039050       8        1   3.938105
15    -0.035836 -0.099077       9        1   2.721784
8      0.024255 -0.068613      10        1   2.314134
13    -0.022009 -0.083263      11        1   2.310481
16    -0.011045 -0.033132      12        1   2.218512
19    -0.057814 -0.062302      13        1   2.063104
6     -0.077469 -0.032859      14        1   1.885814
4     -0.026847  0.218965      15        1   1.627359
2     -0.156735  0.013776      16        1   1.280559
18    -0.198384 -0.001930      17        1   1.125492
14    -0.235958  0.000095      18        1   0.728163
7     -0.225462 -0.013672      19        1   0.702333
10    -0.239001  0.003655      20        1   0.168525, topic_info=      Term         Freq        Total Category  logprob  loglift
5     data  1555.000000  1555.000000  Default  30.0000  30.0000
70      of   592.000000   592.000000  Default  29.0000  29.0000
144    the   490.000000   490.000000  Default  28.0000  28.0000
452      )   164.000000   164.000000  Default  27.0000  27.0000
9       in   621.000000   621.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
20     day     0.016425    21.433077  Topic20  -7.6971  -0.7881
22    free     0.016425    24.064251  Topic20  -7.6971  -0.9039
25   night     0.016425    10.463790  Topic20  -7.6971  -0.0710
27      on     0.016425   207.793001  Topic20  -7.6971  -3.0597
28     one     0.016425     1.046513  Topic20  -7.6971   2.2314

[883 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
554      18  0.681962            &amp
529       3  0.555559           (9:00
105       9  0.962448              (e
78        3  0.985727        (limited
170       4  0.958112  (office-based)
...     ...       ...             ...
316       2  0.956032            your
480      19  0.841195               –
254       1  0.808408               …
254       2  0.010233               …
254       6  0.180783               …

[586 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 4, 12, 2, 10, 13, 18, 16, 9, 14, 17, 20, 7, 5, 3, 19, 15, 8, 11])

In [17]:
p.topic_info[p.topic_info["Category"] == "Topic1"]["Term"].to_list()

['administrative',
 'integrity',
 'analytics',
 'models',
 'complex',
 'cleaning',
 'degree',
 'any',
 'maintain',
 'support',
 'campaigns',
 'ability',
 'interpret',
 'quality',
 'general',
 'performance',
 'through',
 'visualization',
 'study',
 'course',
 'be',
 'data-driven',
 'key',
 'optimize',
 'with…',
 'teams',
 'sales',
 'software',
 'metrics',
 'clerical',
 'will',
 'accurate',
 'insights',
 '…',
 'as',
 'data',
 'accuracy',
 'and',
 'the',
 'by',
 'processes',
 'information',
 'to',
 'for',
 'with',
 'marketing',
 'of',
 'in',
 'entry',
 'on',
 'a',
 'analysis']

In [18]:
coherence_model_lda = CoherenceModel(model=lda_modoel, texts=df_word_comments_cleaned, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3787498284266959


In [19]:
df["snippet_list"] = [re.split(r"[,\s\t;.\n]+",sentence) for sentence  in df["snippet"]].copy()

In [36]:
topics = p.topic_info[p.topic_info["Category"] == "Topic1"]["Term"].to_list()
remove_words = ['…',
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    'through',
    'if',
    'as',
    '*',
    'have',
    'course',
    'at',
    'are',
    'for…',
    '(40%)',
    'analysis?',
    'you',
    'that',
    'a',
    'in',
    'with',
    'and',
    'of',
    'to',
    'is',
    'be',
    'by',
    'will',
    'any',
    'on',
    'for',
    'the',
    'with…']
to_check = [word for word in topics if word not in remove_words]
to_check

['administrative',
 'integrity',
 'analytics',
 'models',
 'complex',
 'cleaning',
 'degree',
 'maintain',
 'support',
 'campaigns',
 'ability',
 'interpret',
 'quality',
 'general',
 'performance',
 'visualization',
 'study',
 'data-driven',
 'key',
 'optimize',
 'teams',
 'sales',
 'software',
 'metrics',
 'clerical',
 'accurate',
 'insights',
 'data',
 'accuracy',
 'processes',
 'information',
 'marketing',
 'entry',
 'analysis']

In [37]:
df["extractedSalary/type"].unique()

array(['monthly', 'hourly', 'yearly', 'daily', 'weekly'], dtype=object)

In [38]:
df.columns

Index(['company', 'companyOverviewLink', 'companyRating', 'companyReviewCount',
       'normTitle', 'extractedSalary/max', 'extractedSalary/min',
       'extractedSalary/type', 'formattedLocation', 'formattedRelativeTime',
       'jobDescription', 'snippet', 'taxonomyAttributes/1/attributes/0/label',
       'taxonomyAttributes/4/attributes/0/label', 'viewJobLink',
       'snippet_list'],
      dtype='object')

In [39]:
df_info = df.loc[:,['extractedSalary/max', 'extractedSalary/min', 'extractedSalary/type', 'snippet']].copy()
df_info.head()

,extractedSalary/max,extractedSalary/min,extractedSalary/type,snippet
0,25000.0,20000.0,monthly,Perform data entry and scan documents.\n Provi...
1,29000.0,20000.0,monthly,HMO with 2 free beneficiaries on Day One.\n Sh...
2,70000.0,70000.0,monthly,Have personally led and implemented initiative...
3,55000.0,40000.0,monthly,This is minimum of 12 Months Project Based emp...
4,30000.0,18000.0,monthly,Non-Voice Agent (limited slots).\n Amenable to...


In [40]:
df_info["snippet_list"] = [re.split(r"[,\s\t;.\n]+",sentence) for sentence  in df_info["snippet"]].copy()
df_info.head()

,extractedSalary/max,extractedSalary/min,extractedSalary/type,snippet,snippet_list
0,25000.0,20000.0,monthly,Perform data entry and scan documents.\n Provi...,"[Perform, data, entry, and, scan, documents, P..."
1,29000.0,20000.0,monthly,HMO with 2 free beneficiaries on Day One.\n Sh...,"[HMO, with, 2, free, beneficiaries, on, Day, O..."
2,70000.0,70000.0,monthly,Have personally led and implemented initiative...,"[Have, personally, led, and, implemented, init..."
3,55000.0,40000.0,monthly,This is minimum of 12 Months Project Based emp...,"[This, is, minimum, of, 12, Months, Project, B..."
4,30000.0,18000.0,monthly,Non-Voice Agent (limited slots).\n Amenable to...,"[Non-Voice, Agent, (limited, slots), Amenable,..."


In [41]:
sentence_list = list()
item_topic_bool= list()
for array in df_info["snippet_list"]:
    topic_list_arr = list()
    item_count = 0
    for items in array:
        if items in to_check:
            topic_list_arr.append(True)
        else:
            topic_list_arr.append(False)
    if True in topic_list_arr:
        sentence_list.append(np.sum(topic_list_arr))
        item_topic_bool.append(True)
    else:
        sentence_list.append(np.sum(topic_list_arr))
        item_topic_bool.append(False)
        

In [42]:
df_info["Topic1?"] = item_topic_bool
df_info["Topic1Count"] = sentence_list

In [43]:
df_info

,extractedSalary/max,extractedSalary/min,extractedSalary/type,snippet,snippet_list,Topic1?,Topic1Count
0,25000.0,20000.0,monthly,Perform data entry and scan documents.\n Provi...,"[Perform, data, entry, and, scan, documents, P...",True,7
1,29000.0,20000.0,monthly,HMO with 2 free beneficiaries on Day One.\n Sh...,"[HMO, with, 2, free, beneficiaries, on, Day, O...",False,0
2,70000.0,70000.0,monthly,Have personally led and implemented initiative...,"[Have, personally, led, and, implemented, init...",False,0
3,55000.0,40000.0,monthly,This is minimum of 12 Months Project Based emp...,"[This, is, minimum, of, 12, Months, Project, B...",False,0
4,30000.0,18000.0,monthly,Non-Voice Agent (limited slots).\n Amenable to...,"[Non-Voice, Agent, (limited, slots), Amenable,...",False,0
...,...,...,...,...,...,...,...
1116,49000.0,44000.0,monthly,Strong attention to detail and accuracy in fin...,"[Strong, attention, to, detail, and, accuracy,...",True,4
1117,35000.0,25000.0,monthly,"Proficiency in database design, data modeling,...","[Proficiency, in, database, design, data, mode...",True,6
1118,-1.0,525.0,daily,Maintain and update existing data records as n...,"[Maintain, and, update, existing, data, record...",True,2
1119,-1.0,100000.0,monthly,"Experience with data warehousing, data archite...","[Experience, with, data, warehousing, data, ar...",True,5


In [44]:
print("the key words are the ff ",to_check)
df_info.drop(["snippet","snippet_list"],axis=1).groupby(["Topic1?","Topic1Count","extractedSalary/type"]).mean()

the key words are the ff  ['administrative', 'integrity', 'analytics', 'models', 'complex', 'cleaning', 'degree', 'maintain', 'support', 'campaigns', 'ability', 'interpret', 'quality', 'general', 'performance', 'visualization', 'study', 'data-driven', 'key', 'optimize', 'teams', 'sales', 'software', 'metrics', 'clerical', 'accurate', 'insights', 'data', 'accuracy', 'processes', 'information', 'marketing', 'entry', 'analysis']


extractedSalary/max  \
Topic1? Topic1Count extractedSalary/type                        
False   0           daily                          693.333333   
                    hourly                         100.153846   
                    monthly                      33198.943953   
True    1           hourly                         150.750000   
                    monthly                      43282.306748   
        2           daily                           -1.000000   
                    monthly                      36739.696296   
        3           daily                          183.571429   
                    monthly                      38291.472222   
        4           daily                          645.000000   
                    hourly                          57.500000   
                    monthly                      37134.418033   
                    weekly                          -1.000000   
                    yearly                      664000.000000   
        5           daily                         1500.000000   
                    monthly                      37660.101351   
        6           monthly                      56370.333333   
        7           monthly                      49617.588235   
        8           monthly                      53610.944444   

                                          extractedSalary/min  
Topic1? Topic1Count extractedSalary/type                       
False   0           daily                          693.333333  
                    hourly                         150.846154  
                    monthly                      24288.539823  
True    1           hourly                         266.170000  
                    monthly                      30640.110429  
        2           daily                          683.333333  
                    monthly                      28897.981481  
        3           daily                          460.714286  
                    monthly                      33922.526319  
        4           daily                          645.000000  
                    hourly                         116.666667  
                    monthly                      34139.344262  
                    weekly                        7400.000000  
                    yearly                      531000.000000  
        5           daily                         1000.000000  
                    monthly                      27998.087838  
        6           monthly                      39370.370370  
        7           monthly                      39323.529412  
        8           monthly                      41944.444444